In [4]:
import streamlit_functions as sf
import importlib as imp
import geopandas as gpd
import matplotlib.pyplot as plt
import functions as f
# debugging
import importlib as imp
imp.reload(f)

# graph
import networkx as nx
import h3
from functions import H3_INDEX_RESOLUTION
from shapely.geometry import Point

### find closest bikelines based on the h3 index

In [5]:
bikelane_all = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_WGS84_h3_indexed.parquet')

In [7]:
coordinates1 = [[11.581726, 55.606109], [11.699829, 55.592143]] # sjeland
coordinates2 = [[12.473907, 55.717571], [12.561798, 55.695132]] # in Copenhagen
coordinates3 = [[12.200317, 55.864524], [11.881714, 55.523967]] # big distance

In [166]:
# Assuming 'bikelane_all' is your GeoDataFrame of bike lanes
# and 'coordinates2' are your start and end points in Copenhagen

# Convert points to Shapely Points
start_point = Point(coordinates3[0])
end_point = Point(coordinates3[1])


# Function to find the closest LineString to a point, considering H3 index
def find_closest_objects(point, gdf):
    point_h3_index = f.get_h3_index_from_point(point)
    # Filter linestrings by H3 index
    filtered_gdf = gdf[gdf['h3_index'] == point_h3_index]
    if len(filtered_gdf) == 0:
        # If no lines are found in the same H3 index, search in the neighboring indexes
        neighbors = h3.k_ring(point_h3_index, 1)
        filtered_gdf = gdf[gdf['h3_index'].isin(neighbors)]
    
    min_dist = float('inf')
    closest_line = None
    for row in filtered_gdf.itertuples():
        # Calculate the distance from the point to the current line
        dist = point.distance(row.geometry)
        if dist < min_dist:
            min_dist = dist
            closest_line = row.geometry
            closest_edge = row.Index  # itertuples gives us the index in the 'Index' attribute
            closest_line_point = f.closest_coordinate_on_linestring(point, closest_line)
    return closest_line, closest_edge[0:2], closest_line_point, filtered_gdf

# Find closest LineStrings to start and end points

closest_line_start, closest_edge_start, closest_line_point_start, filtered_gdf_1 = find_closest_objects(start_point, bikelane_all)
closest_line_end, closest_edge_end, closest_line_point_end, filtered_gdf_2 = find_closest_objects(end_point, bikelane_all)
print(closest_line_start, closest_edge_start, closest_line_point_start)
print(closest_line_end, closest_edge_end, closest_line_point_end)


LINESTRING (12.2003564 55.8647522, 12.1994907 55.8648599, 12.1990102 55.8647479) (1756628177, 10138199326) POINT (12.2003564 55.8647522)
LINESTRING (11.8775351 55.5236505, 11.8801196 55.5228341, 11.8802918 55.522796, 11.8805005 55.5227833, 11.882419 55.523005, 11.8826094 55.5229762, 11.8826692 55.5228818, 11.8831514 55.5211698, 11.8832029 55.520987, 11.8832883 55.5209253, 11.8834251 55.5208995, 11.8835136 55.5208707, 11.8835163 55.5207932) (1326937882, 5611255165) POINT (11.882419 55.523005)


### Preparing for shortest path 

In [39]:
# Initialize an empty list to store edges
edge_list = []

# Loop through the DataFrame using itertuples
for row in bikelane_all.itertuples(index=True, name='Edge'):
    # Extract source, target, and other relevant columns from the row tuple
    # print(row[1])
    source = row[0][0]  # Extract u from the multi-index
    target = row[0][1]  # Extract v from the multi-index
    geometry = row[1]
    length = geometry.length
    
    # Append the edge to the edge list
    edge_list.append((source, target, {'geometry': geometry, 'length': length}))

# Display the edge list
# print(edge_list)

In [40]:
G = nx.from_edgelist(edge_list, create_using=nx.DiGraph())

### Shortest Path

In [167]:
path = nx.shortest_path(G, source=closest_edge_start[0], target=closest_edge_end[1], weight='length')

In [168]:
l = []
for vertex in range(len(path)-1):
    u = path[vertex]
    v = path[vertex + 1]
    l.append((u, v, 0))

result = bikelane_all.loc[l]
result.explore()

#### THIS IS IT

In [161]:
l

[(283279508, 283279324, 0),
 (283279324, 1660681702, 0),
 (1660681702, 1906849597, 0),
 (1906849597, 1906849611, 0),
 (1906849611, 1906849599, 0),
 (1906849599, 1906849642, 0),
 (1906849642, 1906849582, 0),
 (1906849582, 1906849637, 0),
 (1906849637, 4082804182, 0),
 (4082804182, 4082804181, 0),
 (4082804181, 4082804183, 0),
 (4082804183, 288425351, 0),
 (288425351, 26949060, 0),
 (26949060, 26949011, 0),
 (26949011, 323027766, 0),
 (323027766, 2350495092, 0),
 (2350495092, 5453709388, 0),
 (5453709388, 299511200, 0),
 (299511200, 385552925, 0),
 (385552925, 385552931, 0),
 (385552931, 300445985, 0),
 (300445985, 2351216341, 0),
 (2351216341, 452546315, 0),
 (452546315, 271804148, 0),
 (271804148, 271804098, 0),
 (271804098, 30443010, 0),
 (30443010, 20930083, 0),
 (20930083, 2398191540, 0),
 (2398191540, 20930084, 0),
 (20930084, 2687267411, 0),
 (2687267411, 9828146651, 0),
 (9828146651, 2687267399, 0),
 (2687267399, 2687267401, 0),
 (2687267401, 1400773123, 0),
 (1400773123, 2991111

 #### Faster index

In [45]:
# Write to GraphML
nx.write_graphml(G_edges, 'graph.graphml')